In [1]:
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import glob
import json

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Find the dataset @ (CSV files 2022-04-21, https://webrobots.io/kickstarter-datasets/)
# download link: https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2022-04-21T03_20_08_060Z.zip

### Create master Data Frame
before running this code, create a folder inside the repo folder called 'raw_data' contiaining the exctracted CSV files from the download link.

The code will create a new directory called 'data' to store the processed data.

In [2]:
# create folder inside the current directory to hold the processed data
try:
    Path('data').mkdir(parents=True, exist_ok=False)
    print ('creating data directory')
except:
    print ('directory already exists')
    pass

directory already exists


In [3]:
# path to each file in the all_data folder
all_paths = glob.glob('raw_data/*.csv')

# list to append df's to
list_of_df = []

# loop through all paths and append each csv as a df
for filename in all_paths:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_of_df.append(df)

# concat all df's into one df
master_df = pd.concat(list_of_df, axis=0, ignore_index=True)

# save df as master csv
master_df.to_csv('data/master_df.csv', index=False)

### Unpack columns containing JSON objects
The code below takes all columns that represent dictionaries and saves them as individual csv files.

In [4]:
def make_json(string):
    '''
    converts the string representation of a json object into a python dict.
    returns np.nan if the string contains commas.
    '''
    try:
        # replace single quotations to make the string represent a JSON object
        json_acceptable_string = string.replace("'", "\"")
        return(json.loads(json_acceptable_string))
    except:
        # if the string is still not JSON compatible return np.nan
        return np.nan

In [5]:
def unpack(data_frame, column):
    '''
    unpacks dict in column to dataframe.
    '''
    # convert all strings into JSON objects
    unpacked = data_frame[column].apply(make_json).to_frame()

    # unpack JSON into DataFrame
    return pd.json_normalize(unpacked[column])

In [6]:
# define which columns are represented as dicts
unpack_list = ['category', 'creator', 'location', 'photo', 'profile', 'urls']

# init empty df 
unpacked_df = pd.DataFrame()

# unpack each column of dicts, save each as their own csv
for value in unpack_list:
    frame = unpack(master_df, value)
    frame.to_csv(f'data/{value}.csv', index=False)  

In [7]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]

In [8]:
# drop columns deemed unessecary
model_data = master_df.drop(columns = ['backers_count','usd_pledged','is_starrable','current_currency','static_usd_rate','usd_exchange_rate','usd_type','id','name', 'slug', 'category', 'creator', 'location', 'photo', 'profile', 'urls', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'disable_communication', 'source_url', 'currency', 'pledged', 'blurb'])

# create goal_usd column so that all goal amounts are in the same units, drop fx_rate and goals afterward
model_data['goal_usd'] = model_data['fx_rate']*model_data['goal']
model_data = model_data.drop(columns=['fx_rate', 'goal'])

# combine category data and all other date
model_data = pd.concat([model_data, df_category], axis=1, join='inner')

# create percentage funded column as another metric
model_data['percentage_funded'] = model_data.converted_pledged_amount/model_data.goal_usd*100

# create total days active column as another metric
model_data['total_days_active'] = (model_data.deadline-model_data.created_at)*0.00001157

model_data

,converted_pledged_amount,country,created_at,deadline,launched_at,spotlight,staff_pick,state,state_changed_at,goal_usd,name,parent_name,percentage_funded,total_days_active
0,5034,US,1609376406,1611968831,1609545583,True,False,successful,1611968831,5000.000000,Cookbooks,Food,100.680000,29.994357
1,48365,HK,1606278560,1614096106,1608912106,True,True,successful,1614096106,44624.926500,Cookbooks,Food,108.381131,90.449007
2,98,ES,1606477096,1607900340,1606751262,True,False,successful,1607900340,54.095050,Cookbooks,Food,181.162603,16.466933
3,127765,US,1604500905,1610082068,1606194068,True,True,successful,1610082068,65000.000000,Cookbooks,Food,196.561538,64.574056
4,14574,GB,1605454727,1608573895,1605981895,True,True,successful,1608573895,13823.881138,Cookbooks,Food,105.426254,36.088774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,US,1434905504,1443967460,1441375460,True,False,successful,1443967460,3000.000000,Fiction,Publishing,113.166667,104.846831
30988,21112,US,1440643700,1443937679,1441345679,True,False,successful,1443937681,12000.000000,Fiction,Publishing,175.933333,38.111337
30989,2000,US,1441111356,1446319364,1441135364,True,False,successful,1446319364,2000.000000,Fiction,Publishing,100.000000,60.256653
30990,3125,US,1436479764,1443703429,1441111429,True,False,successful,1443703429,3000.000000,Fiction,Publishing,104.166667,83.577804


In [9]:
model_data = model_data.dropna()

In [48]:
# split into X and y variables
X = model_data.drop(columns=['state'])
y = model_data.state#.to_frame()

In [49]:
# generate 70% train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [50]:
# init encoder for x values
encoder = LabelEncoder()

# encode boolean and string type columns
encoding_columns = (X.select_dtypes(include=['bool', 'object'])).columns
for column in encoding_columns:
    X_train[column] = encoder.fit_transform(X_train[column])
    X_test[column] = encoder.transform(X_test[column])

In [51]:
# new encoder for y variable (to make sure that we can reverse encoding)
y_encoder = LabelEncoder()
y_train = y_encoder.fit_transform(y_train)
y_test = y_encoder.transform(y_test)

In [52]:
# init standard scaler
scaler = StandardScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
X_train_scaled.head()

,converted_pledged_amount,country,created_at,deadline,launched_at,spotlight,staff_pick,state_changed_at,goal_usd,name,parent_name,percentage_funded,total_days_active
0,-0.168986,-2.331170,-1.173276,-1.221472,-1.218599,-1.293287,-0.453535,-1.221367,-0.058870,1.572862,0.040028,-0.039958,-0.326504
1,-0.112866,0.637817,1.495310,1.552086,1.556502,-1.293287,2.204900,1.526362,0.000640,-0.086355,0.813031,-0.038405,0.385631
2,-0.166785,0.637817,0.854143,0.959321,0.933918,-1.293287,-0.453535,0.962323,0.044074,-1.745572,0.040028,-0.039962,0.697305
3,-0.169210,0.637817,-0.967764,-1.044941,-1.019297,-1.293287,-0.453535,-1.044601,-0.071691,-0.915963,-0.346473,-0.037201,-0.514707
4,-0.134489,0.637817,1.230590,1.190614,1.194829,0.773224,-0.453535,1.193923,-0.065958,-0.455070,-1.505977,-0.027661,-0.251157


## Begin Modelling here


In [53]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
classifier = SVC(kernel='linear',max_iter=100000)
classifier

SVC(kernel='linear', max_iter=100000)

In [54]:
# Fit the data
classifier.fit(X_train, y_train)

C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=100000)

In [55]:
# Score the accuracy
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.668144578313253
Testing Data Score: 0.6762622287192174


In [59]:
# Make predictions using the test data
predictions = classifier.predict(X_test)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test
}).reset_index(drop=True)
results.tail()

,Prediction,Actual
8888,3,3
8889,3,3
8890,3,3
8891,1,3
8892,3,3


In [57]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
cm, index=['Actual 0', 'Actual 1', 'Actual 2', 'Actual 3'], columns=['Predicted 0', 'Predicted 1', 'Predicted 2', 'Predicted 3']
)
# Displaying results
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,244,17,1,31
Actual 1,5,1270,0,1559
Actual 2,1,0,228,0
Actual 3,14,1251,0,4272


In [58]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.92      0.83      0.88       293
           1       0.50      0.45      0.47      2834
           2       1.00      1.00      1.00       229
           3       0.73      0.77      0.75      5537

    accuracy                           0.68      8893
   macro avg       0.79      0.76      0.77      8893
weighted avg       0.67      0.68      0.67      8893

